In [3]:
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import os
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
image = Image.open('raw_data/030 ovos_moles/0001.jpeg')

In [10]:
width, height = image.size

In [11]:
width

960

In [ ]:
def initialize_model():
    
    model = models.Sequential()

    model.add(layers.Conv2D(16, kernel_size=(4, 4), activation='relu', padding='same', input_shape=(76, 78, 1)))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))
    model.add(layers.Conv2D(64, kernel_size=(2, 2), activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    model.add(layers.Flatten())
    
    model.add(layers.Dense(10, activation='relu'))

    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model  

In [ ]:
model = initialize_model()

es = EarlyStopping(patience=10)

history = model.fit(X_train, y_train,
                 epochs=100,
                 batch_size=32, 
                 verbose=1,
                 callbacks=[es],
                 validation_split=0.3)

In [ ]:
def plot_loss_accuracy(history, title=None):
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    
    # --- LOSS --- 
    
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('Model loss')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylim((0,3))
    ax[0].legend(['Train', 'Test'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # --- ACCURACY
    
    ax[1].plot(history.history['accuracy'])
    ax[1].plot(history.history['val_accuracy'])
    ax[1].set_title('Model Accuracy')
    ax[1].set_ylabel('Accuracy')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Test'], loc='best')
    ax[1].set_ylim((0,1))
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
    
    if title:
        fig.suptitle(title)